In [13]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [14]:
def fetch(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.content, "html.parser")
    return soup

In [15]:
def get_categories():
    top_category_url = "https://se.trustpilot.com/categories"
    category_soup = fetch(top_category_url)
    tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    data = json.loads(tags.text)
    tree = data["props"]
    all_categories = []
    cats = tree["pageProps"]["categories"]
    for category in cats:
        all_categories.append(category["categoryId"])
    category_urls = []
    for cat in all_categories:
        category_url = top_category_url + "/" + cat
        category_urls.append(category_url) # All category URLs collected from site
    return category_urls

In [16]:
categories = get_categories()

In [17]:
print(categories)

['https://se.trustpilot.com/categories/animals_pets', 'https://se.trustpilot.com/categories/electronics_technology', 'https://se.trustpilot.com/categories/events_entertainment', 'https://se.trustpilot.com/categories/vehicles_transportation', 'https://se.trustpilot.com/categories/business_services', 'https://se.trustpilot.com/categories/health_medical', 'https://se.trustpilot.com/categories/home_garden', 'https://se.trustpilot.com/categories/hobbies_crafts', 'https://se.trustpilot.com/categories/home_services', 'https://se.trustpilot.com/categories/legal_services_government', 'https://se.trustpilot.com/categories/construction_manufactoring', 'https://se.trustpilot.com/categories/food_beverages_tobacco', 'https://se.trustpilot.com/categories/media_publishing', 'https://se.trustpilot.com/categories/money_insurance', 'https://se.trustpilot.com/categories/travel_vacation', 'https://se.trustpilot.com/categories/restaurants_bars', 'https://se.trustpilot.com/categories/public_local_services', 

In [18]:
companies = []

for category in categories[0:2]:
    category_soup = fetch(category)
    category_tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    category_data = json.loads(category_tags.text)
    # Category is for example https://se.trustpilot.com/categories/animals_pets
    n_pages = category_data["props"]["pageProps"]["seoData"]["maxPages"]
    for i in range(1, n_pages):
        category_page = category + "?page=" + str(i)
        company_soup = fetch(category_page)
        company_tags = company_soup.find("script", {"id":"__NEXT_DATA__"})
        company_data = json.loads(company_tags.text)
        company_tree = company_data["props"]["pageProps"]["businessUnits"]
        for k, v in company_tree.items():
            if type(v) == list:
                for j in v:
                    company_id = j["identifyingName"]
                    if company_id not in companies:
                        companies.append(company_id)
        i += 1

In [19]:
print(len(companies))

1345


In [20]:
for c in companies[0:10]: # CHANGE to all companies
    count_soup = fetch("https://se.trustpilot.com/review/" + c)
    count_tags = count_soup.find("script", {"id":"__NEXT_DATA__"})
    count_data = json.loads(count_tags.text)
    count_tree = count_data["props"]
    count_pages = count_tree["pageProps"]["filters"]["pagination"]["totalPages"]
    for page in range(1, count_pages):
        review_soup = fetch("https://se.trustpilot.com/review/" + c + "?page=" + str(page))
        review_tags = review_soup.find("script", {"id":"__NEXT_DATA__"})
        review_data = json.loads(review_tags.text)
        review_tree = review_data["props"]["pageProps"]
        review_tree = review_tree["reviews"]
        review = review_tree[0]["text"]
        rating = review_tree[0]["rating"]

KeyboardInterrupt: 